In [295]:
from __future__ import annotations

data = open("data/22.txt", "r").read()

In [296]:
map_str, instructions_str = data.split("\n\n")

# Part 1

In [297]:
from dataclasses import dataclass


class Node:
    value: str
    coord: tuple[int, int]
    edges: dict[int, Node]

    def __init__(self, value, coord, edges):
        self.value = value
        self.coord = coord
        self.edges = edges

    def __repr__(self):
        return f"{self.value} with neighbors {[f'{edge_index}: {edge_node.value}, ' for edge_index, edge_node in self.edges.items()]}"

In [298]:
grid_map: dict[tuple[int, int], Node] = {}

In [299]:
map_lines = map_str.splitlines()

# Let's pad our rows
max_col_index = 0

for row in map_lines:
    max_col_index = max(len(row) - 1, max_col_index)

for i in range(len(map_lines)):
    map_lines[i] = map_lines[i].ljust(max_col_index + 1, " ")

In [300]:
#  3
# 2*0
#  1

left_top_node = None
col_start_nodes: dict[int, Node] = {}
col_end_nodes: dict[int, Node] = {}
for row_index, row in enumerate(map_lines):
    row_start_node = None
    last_node = None
    for char_index, char in enumerate(row):
        if char == " ":
            if (row_index - 1, char_index) in grid_map:
                col_end_nodes[char_index] = grid_map[(row_index - 1, char_index)]
            continue

        new_node = Node(
            value=char,
            coord=(char_index, row_index),
            edges={}
        )
        grid_map[(row_index, char_index)] = new_node

        # Detect if this is the first node ever
        if row_index == 0 and row_start_node is None:
            left_top_node = new_node

        # Setup row start node
        if row_start_node is None:
            row_start_node = new_node
        # Setup col start node
        if char_index not in col_start_nodes:
            col_start_nodes[char_index] = new_node

        # Patch neighbor node (horizontal)
        if last_node is not None:
            new_node.edges[2] = last_node
            last_node.edges[0] = new_node

        # Patch neighbor node (vertical)
        if (row_index - 1, char_index) in grid_map:
            new_node.edges[3] = grid_map[(row_index - 1, char_index)]
            grid_map[(row_index - 1, char_index)].edges[1] = new_node

        # Detect last col and patch
        if char_index + 1 == len(row) or ((char_index + 1) < len(row) and row[char_index + 1] == " "):
            new_node.edges[0] = row_start_node
            row_start_node.edges[2] = new_node

        # Detect last row and add to col_end_nodes
        if row_index + 1 == len(map_lines):
            col_end_nodes[char_index] = new_node

        # Update last
        last_node = new_node

# Patch columns
for i in range(max_col_index + 1):
    col_start_nodes[i].edges[3] = col_end_nodes[i]
    col_end_nodes[i].edges[1] = col_start_nodes[i]

In [301]:
import re

token_pattern = re.compile(r"(R|L|(?:\d+))")
tokens = token_pattern.findall(instructions_str)
tokens = [int(token) if token not in ("R", "L") else token for token in tokens]

In [302]:
facing_map = {
    0: (1, 0),
    1: (0, -1),
    2: (-1, 0),
    3: (0, 1)
}

In [303]:
turning_map = {"R": 1, "L": -1}

In [304]:
current_node = left_top_node
current_direction = 0
for token in tokens:
    match token:
        case str():
            current_direction = (current_direction + turning_map[token]) % 4
        case int():
            for _ in range(token):
                if current_node.edges[current_direction].value == "#":
                    break
                current_node = current_node.edges[current_direction]

In [305]:
(current_node.coord[1] + 1) * 1000 + (current_node.coord[0] + 1) * 4 + current_direction

95358

# Part 2